In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import numpy as np
import imgaug.augmenters as iaa


#/content/drive/My Drive/level_0_500
DATADIR = "/content/drive/MyDrive/Augmentation"
CATEGORIES = ["level0_aug","level4_aug"] 

In [ ]:
training_data = []


def create_training_data():
    for category in CATEGORIES :
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img),cv2.IMREAD_COLOR)[...,::-1]
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
                img_array = cv2.addWeighted(img_array,4, cv2.GaussianBlur(img_array , (0,0) , 20) ,-4 ,128)
                new_array = cv2.resize(img_array, (512, 512))
                training_data.append([new_array, class_num])
                rotate=iaa.Affine(rotate=(90))
                rotated_image=rotate.augment_image(new_array)
                training_data.append([rotated_image, class_num])
                flip_hr=iaa.Fliplr(p=1.0)
                flip_hr_image= flip_hr.augment_image(new_array)
                training_data.append([flip_hr_image, class_num])
            except Exception as e:
                pass

In [ ]:
create_training_data()

In [ ]:
X = [] #features
y = [] #labels

for features in training_data:
    X.append(features[0])
    y.append(features[1])
X = np.array(X)
X.shape

(627, 512, 512)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_full, test_X, y_train_full, test_y = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
X_valid, X_train = X_train_full[:70] / 255.0, X_train_full[70:] / 255.0
y_valid, y_train = y_train_full[:70], y_train_full[70:]

In [ ]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [ ]:
X_train.shape,X_valid.shape,y_train.shape

((431, 512, 512), (70, 512, 512), (431,))

In [ ]:
from tensorflow import keras
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[512, 512]))
model.add(keras.layers.Dense(4096, activation="relu"))
model.add(keras.layers.Dense(1024, activation="tanh"))
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(256, activation="tanh"))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(64, activation="tanh"))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(2, activation="sigmoid"))

In [ ]:
sgd = keras.optimizers.SGD(lr=0.001)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=7,batch_size= 10,validation_data=(X_valid, y_valid))

Epoch 1/7
44/44 [==============================] - 6s 127ms/step - loss: 0.6984 - accuracy: 0.4896 - val_loss: 0.7136 - val_accuracy: 0.4714
Epoch 2/7
44/44 [==============================] - 6s 127ms/step - loss: 0.6968 - accuracy: 0.5012 - val_loss: 0.6907 - val_accuracy: 0.5286
Epoch 3/7
44/44 [==============================] - 6s 128ms/step - loss: 0.6961 - accuracy: 0.4849 - val_loss: 0.6938 - val_accuracy: 0.5286
Epoch 4/7
44/44 [==============================] - 6s 126ms/step - loss: 0.6980 - accuracy: 0.4872 - val_loss: 0.6908 - val_accuracy: 0.5286
Epoch 5/7
44/44 [==============================] - 6s 125ms/step - loss: 0.6933 - accuracy: 0.5128 - val_loss: 0.6889 - val_accuracy: 0.5286
Epoch 6/7
44/44 [==============================] - 6s 126ms/step - loss: 0.6953 - accuracy: 0.4826 - val_loss: 0.6884 - val_accuracy: 0.5286
Epoch 7/7
44/44 [==============================] - 6s 126ms/step - loss: 0.6901 - accuracy: 0.5267 - val_loss: 0.7260 - val_accuracy: 0.4714


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(test_X)

ypred=[]

for i in y_pred:
    if i[0]>i[1]:
        ypred.append(0)
    else:
        ypred.append(1)
ypred=np.array(ypred)

print(confusion_matrix(test_y,ypred))
print(classification_report(test_y,ypred))

[[ 0 63]
 [ 0 63]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        63
           1       0.50      1.00      0.67        63

    accuracy                           0.50       126
   macro avg       0.25      0.50      0.33       126
weighted avg       0.25      0.50      0.33       126



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def create_model():
	# create model
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=[512, 512]))
  model.add(keras.layers.Dense(512, activation="relu"))
  model.add(keras.layers.Dense(512, activation="relu"))
  model.add(keras.layers.Dense(256, activation="relu"))
  model.add(keras.layers.Dense(256, activation="relu"))
  model.add(keras.layers.Dense(128, activation="relu"))
  model.add(keras.layers.Dense(2, activation="sigmoid"))
	# Compile model
 


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy
  sgd = keras.optimizers.SGD(lr=0.0001)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
batch_size = [10, 20, 30, 5, 15, 25]
epochs = [10, 20, 30,5,40]
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

NameError: ignored